<a href="https://colab.research.google.com/github/kimjs2513/TensorFlow/blob/master/008_Fancy_softmax_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
import numpy as np

tf.random.set_seed(777)  # for reproducibility


xy = np.loadtxt('/content/drive/My Drive/csv/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:,[ -1]]

nb_classes = 7  # 0 ~ 6


# Make Y data as onehot shape, 맨 끝 열을 보면 one-hot 형태로 되어 있지 않기 때문에 
Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes) #one hot encoding을 하면 1차원이 늘게 됨
Y_one_hot = tf.reshape(Y_one_hot,[ -1, nb_classes]) #3차원이 된 것을 다시 2차원으로 만들어줘야함. -1은 everything을 의미한다.


print(x_data.shape, Y_one_hot.shape)

#Weight and bias setting
W = tf.Variable(tf.random.normal((16, nb_classes)), name='weight') #x에 관련된 열이 16개, y가 7가지로 출력될 수 있기때문에->nb_classes
b = tf.Variable(tf.random.normal((nb_classes,)), name='bias')
variables = [W, b]

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def logit_fn(X):
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_fn(X))

def cost_fn(X, Y):
    logits = logit_fn(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true=Y, y_pred=logits, 
                                                      from_logits=True)    
    cost = tf.reduce_mean(cost_i)    
    return cost

def grad_fn(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_fn(X, Y)
        grads = tape.gradient(loss, variables)
        return grads
    
def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1) #argmax -> hypothesis를 0~6사이의 값으로 만들어준다.
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32)) #cast는 파라미터가 float일 경우 소수점을 버림. boolean일 경우에는 true or false를 return한다.

    return accuracy

def fit(X, Y, epochs=1000, verbose=100):
    optimizer =  tf.keras.optimizers.SGD(learning_rate=0.1)

    for i in range(epochs):
        grads = grad_fn(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            acc = prediction(X, Y).numpy()
            loss = cost_fn(X, Y).numpy() 
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))

fit(x_data, Y_one_hot)



(101, 16) (101, 7)
Steps: 1 Loss: 3.635028839111328, Acc: 0.1683168262243271
Steps: 100 Loss: 0.5194157958030701, Acc: 0.7920792102813721
Steps: 200 Loss: 0.31850090622901917, Acc: 0.9108911156654358
Steps: 300 Loss: 0.23534876108169556, Acc: 0.9405940771102905
Steps: 400 Loss: 0.18872135877609253, Acc: 0.9504950642585754
Steps: 500 Loss: 0.1584603488445282, Acc: 0.9504950642585754
Steps: 600 Loss: 0.13703754544258118, Acc: 0.9900990128517151
Steps: 700 Loss: 0.12098980695009232, Acc: 0.9900990128517151
Steps: 800 Loss: 0.10847963392734528, Acc: 1.0
Steps: 900 Loss: 0.09843039512634277, Acc: 1.0
Steps: 1000 Loss: 0.09016557037830353, Acc: 1.0
